In [0]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://www-us.apache.org/dist/spark/spark-2.4.5/spark-2.4.5-bin-hadoop2.7.tgz
!tar xf spark-2.4.5-bin-hadoop2.7.tgz
!pip install -q findspark

In [0]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.5-bin-hadoop2.7"

In [0]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [5]:
#Importing the data as a pandas dataframe and printing its shape

import pandas as pd
amazon_df = pd.read_csv("/content/Amazon_Responded_Oct05.csv") 
amazon_df.head()
amazon_df.shape

(462029, 25)

In [6]:
# Removing all Null values from the dataframe amazon_df and printing the new dimensions

amazon_clean = amazon_df.dropna(how="all")
amazon_clean.shape

(378134, 25)

In [7]:
#Splitting the tweet_created_at column using "Space" as a delimiter to get the month and date
DateField = amazon_clean["tweet_created_at"].str.split(" ", n = 6, expand = True) 
  
amazon_clean["WeekDay"]= DateField[0] 
amazon_clean["Month"]= DateField[1] 
amazon_clean["Date"]= DateField[2] 
amazon_clean["TimeStamp"]= DateField[3] 
amazon_clean["Geo"]= DateField[4] 
amazon_clean["Year"]= DateField[5] 



/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_i

In [8]:
#Creating Create_Date column by joining the columns "Month" and "Date" in 'mmm dd' format

amazon_clean['Create_Date'] = amazon_clean[['Month', 'Date']].apply(lambda x: ' '.join(x), axis=1)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [9]:
# Printing amazon_clean to check the new column "Create_Date"
amazon_clean.head(5)

,id_str,tweet_created_at,user_screen_name,user_id_str,user_statuses_count,user_favourites_count,user_protected,user_listed_count,user_following,user_description,user_location,user_verified,user_followers_count,user_friends_count,user_created_at,tweet_language,text_,favorite_count,favorited,in_reply_to_screen_name,in_reply_to_status_id_str,in_reply_to_user_id_str,retweet_count,retweeted,text,WeekDay,Month,Date,TimeStamp,Geo,Year,Create_Date
1,'793270689780203520',Tue Nov 01 01:57:25 +0000 2016,SeanEPanjab,143515471.0,51287.0,4079.0,False,74.0,False,Content marketer; Polyglot; Beard aficionado; ...,غریب الوطن,False,1503.0,850.0,Thu May 13 17:43:52 +0000 2010,en,@AmazonHelp Can you please DM me? A product I ...,0.0,False,AmazonHelp,NaN,85741735.0,0.0,False,NaN,Tue,Nov,01,01:57:25,+0000,2016,Nov 01
2,'793281386912354304',Tue Nov 01 02:39:55 +0000 2016,AmazonHelp,85741735.0,2225450.0,11366.0,False,796.0,False,We answer Amazon support questions 7 days a we...,NaN,True,149569.0,53.0,Wed Oct 28 04:17:54 +0000 2009,en,"@SeanEPanjab I'm sorry, we're unable to DM you...",0.0,False,SeanEPanjab,7.930000e+17,143515471.0,0.0,False,NaN,Tue,Nov,01,02:39:55,+0000,2016,Nov 01
3,'793501578766319616',Tue Nov 01 17:14:53 +0000 2016,SeanEPanjab,143515471.0,51287.0,4079.0,False,74.0,False,Content marketer; Polyglot; Beard aficionado; ...,غریب الوطن,False,1503.0,850.0,Thu May 13 17:43:52 +0000 2010,en,@AmazonHelp It was purchased on https://t.co/g...,0.0,False,AmazonHelp,7.930000e+17,85741735.0,0.0,False,@AmazonHelp It was purchased on https://t.co/g...,Tue,Nov,01,17:14:53,+0000,2016,Nov 01
4,'793501657346682880',Tue Nov 01 17:15:12 +0000 2016,SeanEPanjab,143515471.0,51287.0,4079.0,False,74.0,False,Content marketer; Polyglot; Beard aficionado; ...,غریب الوطن,False,1503.0,850.0,Thu May 13 17:43:52 +0000 2010,en,"@AmazonHelp I am following you now, if it help...",0.0,False,AmazonHelp,7.930000e+17,85741735.0,0.0,False,NaN,Tue,Nov,01,17:15:12,+0000,2016,Nov 01
5,'793502854459879424',Tue Nov 01 17:19:57 +0000 2016,AmazonHelp,85741735.0,2225450.0,11366.0,False,796.0,False,We answer Amazon support questions 7 days a we...,NaN,True,149569.0,53.0,Wed Oct 28 04:17:54 +0000 2009,en,@SeanEPanjab Please give us a call/chat so we ...,0.0,False,SeanEPanjab,7.940000e+17,143515471.0,0.0,False,@SeanEPanjab Please give us a call/chat so we ...,Tue,Nov,01,17:19:57,+0000,2016,Nov 01


In [10]:
# Converting Pandas Dataframe "amazon_clean" into Spark Dataframe "Tweet_sdf" and printing the first 10 rows

amazon_clean = amazon_clean.astype(str) # Converting pandas df to string first
Tweet_sdf = spark.createDataFrame(amazon_clean)
Tweet_sdf.show(10, False) # False allows us to show entire content of the columns

+--------------------+------------------------------+----------------+-----------+-------------------+---------------------+--------------+-----------------+--------------+----------------------------------------------------------------------------------------------------------------------------------------------+-------------+-------------+--------------------+------------------+------------------------------+--------------+------------------------------------------------------------------------------------------------------------------------------------------+--------------+---------+-----------------------+-------------------------+-----------------------+-------------+---------+-----------------------------------------------------------------------------------------------------------------------------------------+-------+-----+----+---------+-----+----+-----------+
|id_str              |tweet_created_at              |user_screen_name|user_id_str|user_statuses_count|user_favourites_

In [11]:
# Checking the datatype of our Spark dataframe file Tweet_sdf
print(type(Tweet_sdf))

<class 'pyspark.sql.dataframe.DataFrame'>


In [12]:
# Checking the datatypes of each variable in Tweet_sdf
Tweet_sdf.printSchema()

root
 |-- id_str: string (nullable = true)
 |-- tweet_created_at: string (nullable = true)
 |-- user_screen_name: string (nullable = true)
 |-- user_id_str: string (nullable = true)
 |-- user_statuses_count: string (nullable = true)
 |-- user_favourites_count: string (nullable = true)
 |-- user_protected: string (nullable = true)
 |-- user_listed_count: string (nullable = true)
 |-- user_following: string (nullable = true)
 |-- user_description: string (nullable = true)
 |-- user_location: string (nullable = true)
 |-- user_verified: string (nullable = true)
 |-- user_followers_count: string (nullable = true)
 |-- user_friends_count: string (nullable = true)
 |-- user_created_at: string (nullable = true)
 |-- tweet_language: string (nullable = true)
 |-- text_: string (nullable = true)
 |-- favorite_count: string (nullable = true)
 |-- favorited: string (nullable = true)
 |-- in_reply_to_screen_name: string (nullable = true)
 |-- in_reply_to_status_id_str: string (nullable = true)
 |--

In [13]:
# Extracting columns 'id_str', 'tweet_created_at', 'user_verified', favourite_count', 'retweet_count' and 'text_' for our analysis and printing 20 rows
TweetDataCol = Tweet_sdf.select(Tweet_sdf.id_str, Tweet_sdf.Create_Date, Tweet_sdf.user_verified, Tweet_sdf.favorite_count, Tweet_sdf.retweet_count, Tweet_sdf.text_)
TweetDataCol.show(20, False)

+--------------------+-----------+-------------+--------------+-------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------+
|id_str              |Create_Date|user_verified|favorite_count|retweet_count|text_                                                                                                                                                        |
+--------------------+-----------+-------------+--------------+-------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------+
|'793270689780203520'|Nov 01     |False        |0.0           |0.0          |@AmazonHelp Can you please DM me? A product I ordered last year never arrived.                                                                               |
|'793281386912354304'|Nov 01     |True         |0.0     

In [0]:
# Task 1
# Step 1: Remove the records where “user_verified” is “FALSE”.
# Using Filter function to remove the rows with “user_verified” as “FALSE”

TweetDatafilter = TweetDataCol.rdd.filter(lambda TweetDataCol:'True' in  TweetDataCol[2])

In [15]:
# Task 1 # Step 1
# Printing 5 rows of TweetDatafilter to check the filter
TweetDatafilter.take(5)

[Row(id_str="'793281386912354304'", Create_Date='Nov 01', user_verified='True', favorite_count='0.0', retweet_count='0.0', text_="@SeanEPanjab I'm sorry, we're unable to DM you. Was this order purchased on https://t.co/nUUp5MLhYl, or one of our other sites? ^CL"),
 Row(id_str="'793502854459879424'", Create_Date='Nov 01', user_verified='True', favorite_count='0.0', retweet_count='0.0', text_='@SeanEPanjab Please give us a call/chat so we can look into this order for you: https://t.co/hApLpMlfHN. ^HB'),
 Row(id_str="'793504235400884224'", Create_Date='Nov 01', user_verified='True', favorite_count='0.0', retweet_count='0.0', text_='@SeanEPanjab Without providing any sensitive information, do the order details indicate that the item was ever shipped? ^SW'),
 Row(id_str="'793513446633533440'", Create_Date='Nov 01', user_verified='True', favorite_count='0.0', retweet_count='0.0', text_="@SeanEPanjab I'm not able to access account info here. Please reach out by Phone/Chat so we can look at th

In [16]:
# Task 1 # Step 1
# Obtaining the number of rows from the filtered dataset TweetDatafilter

TweetDatafilter.count()

171899

In [17]:
#Converting the resultant pipelined RDD to Spark Dataframe 

from pyspark.sql import SparkSession
from pyspark import SparkContext
#sc = SparkContext()
#tweets = spark.sparkContext.parallelize(TweetDatafilter.collect())
TweetDatafilter = TweetDatafilter.toDF()
type(TweetDatafilter)

pyspark.sql.dataframe.DataFrame

In [0]:
# Task 1 
# Step 2: For the remaining records (“user_verified” is “TRUE”), group by Create_date, and count the number of tweets for each Create_Date.

import pyspark.sql.functions as f
groupeByDate = TweetDatafilter.groupby('Create_Date').count().withColumnRenamed("count","tweet_count")

In [19]:
# Task 1 # Step 2
# Sorting the result of groupby operation to arrange in ascending order and showing the first 100 rows
# We obtain the highest number of tweets for the date "Jan 03" and the number of tweets are 1536

groupeByDate.sort((groupeByDate.tweet_count).desc()).show(100)

+-----------+-----------+
|Create_Date|tweet_count|
+-----------+-----------+
|     Jan 03|       1536|
|     Jan 10|       1508|
|     Jan 11|       1496|
|     Jan 12|       1410|
|     Jan 06|       1364|
|     Jan 07|       1360|
|     Jan 20|       1342|
|     Mar 02|       1298|
|     Jan 13|       1295|
|     Jan 21|       1292|
|     Jan 14|       1290|
|     Jan 18|       1286|
|     Dec 15|       1279|
|     Jan 24|       1259|
|     Nov 18|       1249|
|     Dec 03|       1201|
|     Jan 02|       1196|
|     Jun 27|       1192|
|     Jul 04|       1190|
|     Jan 19|       1175|
|     Jan 25|       1163|
|     Jan 23|       1149|
|     Jan 08|       1143|
|     Jan 17|       1124|
|     May 11|       1120|
|     Jul 03|       1112|
|     Mar 30|       1109|
|     Apr 05|       1089|
|     Jan 26|       1083|
|     Jan 27|       1080|
|     Jun 20|       1074|
|     Jan 04|       1071|
|     Feb 22|       1058|
|     Nov 05|       1058|
|     Feb 09|       1054|
|     Jun 23

In [0]:
# Task 1 # Step 3
# Filter the records with the date of highest number of tweets, i.e., Jan 03

Step3Data = TweetDatafilter
Step3Data = Step3Data.rdd.filter(lambda Step3Data:'Jan 03' in  Step3Data[1])


In [21]:
# Displaying first 5 records
Step3Data.take(5)

[Row(id_str="'816139627270721537'", Create_Date='Jan 03', user_verified='True', favorite_count='0.0', retweet_count='0.0', text_="@mcp111 Hey! We'll have this looked into internally. Thanks for your patience. ^HK"),
 Row(id_str="'816357712732819467'", Create_Date='Jan 03', user_verified='True', favorite_count='0.0', retweet_count='0.0', text_="@mcp111 Hi there, you can change the country from the settings. Please reach us via the link provided earlier and we'll help you out, ^VM"),
 Row(id_str="'816167583263162369'", Create_Date='Jan 03', user_verified='True', favorite_count='0.0', retweet_count='0.0', text_="@stevebrowne Hi Steve, sorry to hear this. As we can't view accounts here, please contact here for more options: https://t.co/JzP7hlA23B ^CJ"),
 Row(id_str="'816326113660174336'", Create_Date='Jan 03', user_verified='True', favorite_count='0.0', retweet_count='0.0', text_="@boozysmurf We'd like the chance to look into this w/ you &amp; investigate further. Pls reach out by phone/c

In [22]:
# Task 1 # Step 3
# Converting Step3Data to Spark dataframe
Step3Data = Step3Data.toDF()
type(Step3Data)

pyspark.sql.dataframe.DataFrame

In [0]:
# Task 1 # Step 3
# Converting columns user_favourites_count and retweet_count from string data type to integer data type

from pyspark.sql.types import IntegerType
Step3Data = Step3Data.withColumn("favorite_count", Step3Data["favorite_count"].cast(IntegerType()))
Step3Data = Step3Data.withColumn("retweet_count", Step3Data["retweet_count"].cast(IntegerType()))


In [24]:
# Checking the datatypes of columns
Step3Data.printSchema()

root
 |-- id_str: string (nullable = true)
 |-- Create_Date: string (nullable = true)
 |-- user_verified: string (nullable = true)
 |-- favorite_count: integer (nullable = true)
 |-- retweet_count: integer (nullable = true)
 |-- text_: string (nullable = true)



In [0]:
# Task 1 # Step 3
# Calculating the sum of favourite counts and retweet counts for each tweet from Jan 03

Step3Data = Step3Data.withColumn('Sum_favourites_retweets', Step3Data.favorite_count + Step3Data.retweet_count)

In [26]:
# Checking the result by printing 5 rows
Step3Data.take(5)

[Row(id_str="'816139627270721537'", Create_Date='Jan 03', user_verified='True', favorite_count=0, retweet_count=0, text_="@mcp111 Hey! We'll have this looked into internally. Thanks for your patience. ^HK", Sum_favourites_retweets=0),
 Row(id_str="'816357712732819467'", Create_Date='Jan 03', user_verified='True', favorite_count=0, retweet_count=0, text_="@mcp111 Hi there, you can change the country from the settings. Please reach us via the link provided earlier and we'll help you out, ^VM", Sum_favourites_retweets=0),
 Row(id_str="'816167583263162369'", Create_Date='Jan 03', user_verified='True', favorite_count=0, retweet_count=0, text_="@stevebrowne Hi Steve, sorry to hear this. As we can't view accounts here, please contact here for more options: https://t.co/JzP7hlA23B ^CJ", Sum_favourites_retweets=0),
 Row(id_str="'816326113660174336'", Create_Date='Jan 03', user_verified='True', favorite_count=0, retweet_count=0, text_="@boozysmurf We'd like the chance to look into this w/ you &a

In [0]:
# Task 1 # Step 3
# Selecting top 100 tweets with highest sum of favourite_count + retweet_count

from pyspark.sql.functions import desc 
Step3DataTop100 = Step3Data.sort(desc("Sum_favourites_retweets")).limit(100)

In [28]:
# Task 1 # Step 3
# Checking the result by printing first 20 rows
Step3DataTop100.show()

+--------------------+-----------+-------------+--------------+-------------+--------------------+-----------------------+
|              id_str|Create_Date|user_verified|favorite_count|retweet_count|               text_|Sum_favourites_retweets|
+--------------------+-----------+-------------+--------------+-------------+--------------------+-----------------------+
|'816329761530093568'|     Jan 03|         True|             4|            1|@amazon worst sho...|                      5|
|'816083406962434048'|     Jan 03|         True|             2|            1|@ItsJosshA We alw...|                      3|
|'816086117938319360'|     Jan 03|         True|             2|            0|@ItsJosshA Oh no!...|                      2|
|'816095108013654017'|     Jan 03|         True|             1|            1|@KStefl Sounds li...|                      2|
|'816109446069911554'|     Jan 03|         True|             1|            1|@Schoey1992 Happy...|                      2|
|'81615751742852

In [29]:
# Task 1 # Step 3
# Selecting column text_ from Task3DataTop100 to perform word count(map-reduce) for the tweets

Step3Data_tweets = Step3DataTop100.select("text_").rdd.flatMap(lambda x: x).collect()
Step3Data_tweets[0:5]

['@amazon worst shopping  experience,  no service, no substantial reply to complaints, no delivery for 1 week post guarantee date.',
 '@ItsJosshA We always aim to deliver by the date given in your confirmation email. Have we missed that date? Any update in tracking?  ^NF',
 "@ItsJosshA Oh no! I'm sorry! Please reach out to us so that we can look into options: https://t.co/hApLpMlfHN. ^JO",
 '@KStefl Sounds like you know what to add to your Halloween playlist for this year! ^BV',
 '@Schoey1992 Happy Birthday, Matt! #FriendsForever #FriendshipGoals ^JO']

In [0]:
# Creating Spark Context
from pyspark import SparkContext
sc = SparkContext.getOrCreate()

In [31]:
# Task 1 # Step 3
# Reading 'tweet' to a RDD object using spark context
tweet_rdd = sc.parallelize(Step3Data_tweets)
tweet_rdd.take(5)

['@amazon worst shopping  experience,  no service, no substantial reply to complaints, no delivery for 1 week post guarantee date.',
 '@ItsJosshA We always aim to deliver by the date given in your confirmation email. Have we missed that date? Any update in tracking?  ^NF',
 "@ItsJosshA Oh no! I'm sorry! Please reach out to us so that we can look into options: https://t.co/hApLpMlfHN. ^JO",
 '@KStefl Sounds like you know what to add to your Halloween playlist for this year! ^BV',
 '@Schoey1992 Happy Birthday, Matt! #FriendsForever #FriendshipGoals ^JO']

In [32]:
type(tweet_rdd)

pyspark.rdd.RDD

In [0]:
# Task 1 # Step 3
# Cleaning text data to count word frequency 

import string
import re

def clean_tweet(x):
  
  # Delete all the URLs in the tweets
  text00 = re.sub(r'www\S+', '', x)
  text01 = re.sub(r'http\S+', '', text00)
  
  # Delete all the numbers in the tweets
  text1 = ''.join([i for i in text01 if not i.isdigit()])
  
  # Delete all the punctuation marks in the tweets
  text2 = text1.translate(str.maketrans('','',string.punctuation))
  
  # Convert text to LOWERCASE
  text3 = text2.lower()

  return text3

In [35]:
# Task 1 # Step 3
# Cleaning the text of tweets: 1. Removing URLs, 2. Removing non-alphabets, 3. Lowercase 
clean_tweet_rdd = tweet_rdd.map(clean_tweet)
clean_tweet_rdd.take(10)

['amazon worst shopping  experience  no service no substantial reply to complaints no delivery for  week post guarantee date',
 'itsjossha we always aim to deliver by the date given in your confirmation email have we missed that date any update in tracking  nf',
 'itsjossha oh no im sorry please reach out to us so that we can look into options  jo',
 'kstefl sounds like you know what to add to your halloween playlist for this year bv',
 'schoey happy birthday matt friendsforever friendshipgoals jo',
 'ratbones you so fancy you already knooow ep',
 'thorpperrow a happy birthday you dont look a day over   pass on your details here  for lil surprise rs',
 'thedexterouz hi wed like to help when you have a moment please connect with us here   yp',
 'mattlinsley please keep us posted on this and let us know if it doesnt arrive tomorrow bv',
 'vlslt sorry to hear that contact us here  and well look into it for you as']

In [40]:
# Task 1 # Step 3
# Building Map function, printing 20 key-value pairs
map = clean_tweet_rdd.flatMap(lambda line: line.split(" ")).map(lambda word: (word, 1))
map.take(20)

[('amazon', 1),
 ('worst', 1),
 ('shopping', 1),
 ('', 1),
 ('experience', 1),
 ('', 1),
 ('no', 1),
 ('service', 1),
 ('no', 1),
 ('substantial', 1),
 ('reply', 1),
 ('to', 1),
 ('complaints', 1),
 ('no', 1),
 ('delivery', 1),
 ('for', 1),
 ('', 1),
 ('week', 1),
 ('post', 1),
 ('guarantee', 1)]

In [41]:
# Task 1 # Step 3
# Building Reduce function, printing 20 key-value pairs with word count
counts = map.reduceByKey(lambda a, b: a + b)
counts.take(20)

[('amazon', 7),
 ('worst', 1),
 ('', 73),
 ('no', 5),
 ('service', 1),
 ('substantial', 1),
 ('delivery', 6),
 ('week', 1),
 ('itsjossha', 2),
 ('we', 23),
 ('always', 3),
 ('in', 12),
 ('confirmation', 1),
 ('have', 18),
 ('update', 2),
 ('tracking', 5),
 ('oh', 1),
 ('im', 13),
 ('out', 6),
 ('us', 28)]

In [42]:
# Task 1 # Step 3
# Total number of distinct words
print(len(counts.collect()))

608


In [43]:
# Task 1 # Step 3
# Final word count for all words from 100 tweets
countsSorted = counts.takeOrdered(608) # Conditioned on that number of characters in the string should be at least 1
countsSorted

[('', 73),
 ('a', 25),
 ('ab', 1),
 ('abhinavsaroj', 4),
 ('able', 3),
 ('about', 4),
 ('aboutasix', 1),
 ('above', 2),
 ('access', 3),
 ('account', 2),
 ('accttypically', 1),
 ('add', 1),
 ('address', 1),
 ('adibaddy', 1),
 ('advise', 1),
 ('ae', 3),
 ('af', 2),
 ('aim', 2),
 ('ak', 1),
 ('alerts', 2),
 ('alexa', 1),
 ('alexisgraham', 1),
 ('allow', 1),
 ('already', 1),
 ('always', 3),
 ('alyssagoldman', 3),
 ('am', 1),
 ('amazon', 7),
 ('amazonhelp', 1),
 ('amazonin', 2),
 ('ambermullennn', 1),
 ('amp', 1),
 ('an', 7),
 ('and', 17),
 ('any', 8),
 ('anything', 1),
 ('apologies', 2),
 ('app', 6),
 ('application', 1),
 ('appreciation', 1),
 ('ar', 3),
 ('are', 2),
 ('arent', 1),
 ('ariannadelbene', 1),
 ('arrange', 1),
 ('arrive', 6),
 ('arrived', 3),
 ('arrives', 1),
 ('as', 4),
 ('ask', 1),
 ('asked', 1),
 ('assist', 2),
 ('assistance', 1),
 ('at', 6),
 ('athena', 1),
 ('athenarivera', 1),
 ('atoz', 1),
 ('attention', 2),
 ('availability', 1),
 ('available', 2),
 ('aw', 1),
 ('babajab

In [44]:
## Converting the final Map-reduce output into a pandas data frame
import pandas as pd

MapReduceOutput = pd.DataFrame(countsSorted, columns = ['Tweet_Word' , "Frequency"])
print(MapReduceOutput)
MapReduceOutput.head()


       Tweet_Word  Frequency
0                         73
1               a         25
2              ab          1
3    abhinavsaroj          4
4            able          3
..            ...        ...
603          ​hey          1
604             😊          1
605             😌          1
606            😷😷          1
607             🙋          1

[608 rows x 2 columns]


,Tweet_Word,Frequency
0,,73
1,a,25
2,ab,1
3,abhinavsaroj,4
4,able,3


In [0]:
# Exporting output of Map-Reduce output to an excel file

MapReduceOutput.to_excel(r"C:\Users\saksh\UIC 2020 Spring\Analytics for big data\HW2\Ass2_Mapreduce.xlsx")

In [0]:
# Task 2
# Renaming id_str column to tweet_id in Tweet_sdf - our original Amazon_Responded_Oct05.csv Spark dataframe(before filtering operations) 

Tweet_sdf = Tweet_sdf.withColumnRenamed("id_str", "tweet_id")

In [46]:
# Task 2
#Importing the find_text data as a pandas dataframe and printing its shape

import pandas as pd
find_text = pd.read_csv("/content/find_text.csv") 
find_text.head()
find_text.shape

(53928, 2)

In [47]:
# Task 2
# Converting find_text Pandas Dataframe into Spark RDD Dataframe and printing 10 rows

find_text = find_text.astype(str) # Converting pandas df to string first
find_text_sdf = spark.createDataFrame(find_text)
find_text_sdf.show(10, False) # False allows us to show entire content of the columns

+--------------------+----+
|id_str              |text|
+--------------------+----+
|'793270689780203520'|nan |
|'793281386912354304'|nan |
|'793299404975247360'|nan |
|'793301295255945216'|nan |
|'793315815411978240'|nan |
|'793322306848292864'|nan |
|'793322433625415680'|nan |
|'793365409047023616'|nan |
|'793369654878232577'|nan |
|'793375905280393216'|nan |
+--------------------+----+
only showing top 10 rows



In [48]:
# Task 2
# Applying Left join on find_text_sdf with Tweet_sdf as id_str as our key

left_join = find_text_sdf.join(Tweet_sdf, find_text_sdf.id_str == Tweet_sdf.tweet_id, how='left') # Could also use 'left_outer'
left_join.show()

+--------------------+----+--------------------+--------------------+----------------+-----------+-------------------+---------------------+--------------+-----------------+--------------+--------------------+--------------------+-------------+--------------------+------------------+--------------------+--------------+--------------------+--------------+---------+-----------------------+-------------------------+-----------------------+-------------+---------+--------------------+-------+-----+----+---------+-----+----+-----------+
|              id_str|text|            tweet_id|    tweet_created_at|user_screen_name|user_id_str|user_statuses_count|user_favourites_count|user_protected|user_listed_count|user_following|    user_description|       user_location|user_verified|user_followers_count|user_friends_count|     user_created_at|tweet_language|               text_|favorite_count|favorited|in_reply_to_screen_name|in_reply_to_status_id_str|in_reply_to_user_id_str|retweet_count|retweete

In [49]:
# making sure the number of rows in joined dataframe is same as the original find_text dataframe
left_join.count()

53928

In [50]:
# Task 2
# Selecting id_str and text_ from the joined Dataframe left_join

left_join_out = left_join.select(left_join.id_str, left_join.text_)
left_join_out.show(20, False)

+--------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------+
|id_str              |text_                                                                                                                                                     |
+--------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------+
|'793552243689676800'|@AmazonHelp amazon have made and admitted to a series of error that have impacted a myself and a colleague                                                |
|'793595479116439552'|@SuNoSuKo Please don't provide your order details we consider them as personal info. Our twitter page is visible to public. ^VM                           |
|'793751757247619072'|@aasifkhan Hey, I'm sorry for this. Product should reach the fulfillment center, when it

In [51]:
# Converting the Spark dataframe into pandas

join_output = left_join_out.select("*").toPandas()
print(join_output)

                     id_str                                              text_
0      '793552243689676800'  @AmazonHelp amazon have made and admitted to a...
1      '793595479116439552'  @SuNoSuKo Please don't provide your order deta...
2      '793751757247619072'  @aasifkhan Hey, I'm sorry for this. Product sh...
3      '793788930361556996'  @AmazonHelp I would get more assistance from t...
4      '793797330310983680'  Disgusting service @amazonIN\r\nProducts are n...
...                     ...                                                ...
53923  '809401768211685376'  @AmazonVideoIN Censored content, no support fo...
53924  '809412394212466689'  @AmazonHelp @rathipoonam please share details,...
53925  '809449404801421314'  @SlapdashGourmet We'd like to help. Are all it...
53926  '809513226144804864'  @deecaltee When did you reach out to the selle...
53927  '809729921320882179'  @engeetham Hi, sorry for the long stretch. We'...

[53928 rows x 2 columns]


In [0]:
#Exporting the find_text output with id_str and tweet text into an excel file
join_output.to_excel(r"C:\Users\saksh\UIC 2020 Spring\Analytics for big data\HW2\find_text_output.xlsx")